In [1]:
from src.tools.SphericalTensors import SphericalTensor_prolate
from src.molecular_structure.RotationalStates import STM_RotationalBasis, ATM_RotationalBasis
from src.group_theory.Group import *
from src.molecular_structure.TDMs import *
from src.molecular_structure.VibronicStates import VibronicState
from src.quantum_mechanics.Basis import OrthogonalBasis

np.set_printoptions(formatter={'all': lambda x: f"{x.real:.2f}"})

group = C2vGroup()
A1 = C2v_A1_representation(group)
A2 = C2v_A2_representation(group)
B1 = C2v_B1_representation(group)
B2 = C2v_B2_representation(group)

## Coupling angular momentum states
An angular momentum basis can be coupled (not just tensored) with another angular momentum basis. They can also be tensored with other bases (e.g. vibronic basis). The order of operation does not matter.

In [2]:
from src.quantum_mechanics.AngularMomentum import ElectronicSpinBasis, NuclearSpinBasis

# Defining a basis for the vibronic Hilbert space
X = VibronicState("X", symmetry_group=group, irrep = A1)
A = VibronicState("A", symmetry_group=group, irrep= B2)
vibronic_basis = OrthogonalBasis([X,A],"Electronic")

# Basis for the rotational, electronic spin and nuclear spin Hilbert space
rot_basis = STM_RotationalBasis(R_range=(1,1),k_range = (-1,1))
es_basis = ElectronicSpinBasis(S_range=(1/2,1/2))
ns_basis = NuclearSpinBasis(I_range=(1/2,1/2))

# Coupling the rotational and electronic spin bases gives the J basis (Hund's case B)
J_basis = vibronic_basis * rot_basis.couple(es_basis)
J_basis.rename_symbols("J","mJ")

The coupled basis vectors can be expressed in terms of the tensor product basis. The coefficients are given by Clebsch-Gordan coefficients.

In [3]:
J_basis[1].show_composition()

|elec=A, S=0.5, R=1, k=-1, J=0.5, mJ=-0.5> = -0.82 |elec=A, S=0.5, ms=0.5, R=1, k=-1, mR=-1> + 0.58 |elec=A, S=0.5, ms=-0.5, R=1, k=-1, mR=0> 


In [4]:
J_basis[1].quantum_numbers

{'elec': 'A', 'S': 0.5, 'R': 1, 'k': -1, 'J': 0.5, 'mJ': -0.5}

By default, when two bases A and B are coupled to make basis C, show_composition() will write the coupled basis states in terms of the tensor products of the tensor product basis states (a1 x b1, etc.). However, in the case where basis A or (and) B themselves came from coupling simpler angular momentum bases, calling unravel_basis() will write the decomposition of basis vectors in C in terms of the completely unravelled tensor products.

In [5]:
# Coupled the nuclear spin to J to make F.
F_basis = ns_basis.couple(J_basis)
F_basis.rename_symbols("F","mF")

In [5]:
F_basis[2].show_composition()

|elec=X, S=0.5, I=0.5, R=1, k=-1, J=0.5, F=1.0, mF=0.0> = 0.71 |elec=X, S=0.5, I=0.5, mI=-0.5, R=1, k=-1, J=0.5, mJ=0.5> + 0.71 |elec=X, S=0.5, I=0.5, mI=0.5, R=1, k=-1, J=0.5, mJ=-0.5> 


In [6]:
F_basis.unravel_basis()
F_basis[2].show_composition()

|elec=X, S=0.5, I=0.5, R=1, k=-1, J=0.5, F=1.0, mF=0.0> = 0.58 |elec=X, S=0.5, ms=-0.5, I=0.5, mI=-0.5, R=1, k=-1, mR=1> + -0.58 |elec=X, S=0.5, ms=0.5, I=0.5, mI=0.5, R=1, k=-1, mR=-1> + -0.41 |elec=X, S=0.5, ms=0.5, I=0.5, mI=-0.5, R=1, k=-1, mR=0> + 0.41 |elec=X, S=0.5, ms=-0.5, I=0.5, mI=0.5, R=1, k=-1, mR=0> 


## Evaluating transition dipole moments

Dipole moment operators are defined in TDMs.py for

- rotational basis (DipoleOperator_STM)
- vibronic x rotational basis (DipoleOperator_evr)
- spin basis, where it is just the identity matrix (DipoleOperator_spin)

To define the dipole moment operator for your state, take the tensor product of the corresponding operators.

To define dipole operator for a vibronic x rotational state, you need to define the vibronic basis and the rotational basis.

Then, you need to specify what the dipole moment between different vibronic states are (d_a, d_b, d_c), and turn them into a spherical tensor using SphericalTensor_prolate() from tools.SphericalTensors.

Inputting this into DipoleOperator_evr gives you the dipole operator (in space-fixed frame, with q = -1, 0, or 1) for this basis.

In [6]:
X = VibronicState("X", symmetry_group=group, irrep = A1)
A = VibronicState("A", symmetry_group=group, irrep= B2)
vibronic_basis = OrthogonalBasis([X,A],"Electronic")
evr_basis = vibronic_basis * rot_basis

vibronic_d_a = Operator(vibronic_basis,np.array([[0,0],[0,0]]),symmetry_group=group, irrep= A1)
vibronic_d_b = Operator(vibronic_basis,np.array([[0,1],[1,0]]),symmetry_group=group, irrep= B2)
vibronic_d_c = Operator(vibronic_basis,np.array([[0,0],[0,0]]),symmetry_group=group, irrep= B1)

# the transition dipole moment in molecule frame as a spherical tensor
vibronic_d = SphericalTensor_prolate(np.array([vibronic_d_a,vibronic_d_b,vibronic_d_c]),is_operator=True, operator_basis=vibronic_basis)

d_evr = DipoleOperator_evr(evr_basis,vibronic_d, 1)

In [7]:
d_evr.get_connected_states()

[(|elec=X, R=1, k=-1, mR=0>, |elec=A, R=1, k=0, mR=-1>),
 (|elec=X, R=1, k=0, mR=-1>, |elec=A, R=1, k=-1, mR=0>),
 (|elec=X, R=1, k=-1, mR=1>, |elec=A, R=1, k=0, mR=0>),
 (|elec=X, R=1, k=0, mR=0>, |elec=A, R=1, k=-1, mR=1>),
 (|elec=X, R=1, k=0, mR=0>, |elec=A, R=1, k=-1, mR=-1>),
 (|elec=X, R=1, k=0, mR=0>, |elec=A, R=1, k=1, mR=-1>),
 (|elec=X, R=1, k=-1, mR=-1>, |elec=A, R=1, k=0, mR=0>),
 (|elec=X, R=1, k=1, mR=-1>, |elec=A, R=1, k=0, mR=0>),
 (|elec=X, R=1, k=0, mR=1>, |elec=A, R=1, k=-1, mR=0>),
 (|elec=X, R=1, k=0, mR=1>, |elec=A, R=1, k=1, mR=0>),
 (|elec=X, R=1, k=-1, mR=0>, |elec=A, R=1, k=0, mR=1>),
 (|elec=X, R=1, k=1, mR=0>, |elec=A, R=1, k=0, mR=1>),
 (|elec=X, R=1, k=1, mR=0>, |elec=A, R=1, k=0, mR=-1>),
 (|elec=X, R=1, k=0, mR=-1>, |elec=A, R=1, k=1, mR=0>),
 (|elec=X, R=1, k=1, mR=1>, |elec=A, R=1, k=0, mR=0>),
 (|elec=X, R=1, k=0, mR=0>, |elec=A, R=1, k=1, mR=1>)]

If you want to use a basis where rotation is coupled with electron spin or nuclear spin, simply declare those bases and tensor them with the evr basis.

In [8]:
d_es = DipoleOperator_spin(es_basis,1)
d_es.get_connected_states()

[(|S=0.5, ms=-0.5>, |S=0.5, ms=-0.5>), (|S=0.5, ms=0.5>, |S=0.5, ms=0.5>)]

In [9]:
caseB_basis = evr_basis.couple(es_basis)
caseB_basis.rename_symbols("J","mJ")

Since the TDM operator is only defined for the product state and not the coupled state, in order to get the TDM operator for the coupled basis, we need to:

- Define the operator with the uncoupled basis (evr_basis x es_basis), d_uncoupled.
- Use the basis-change matrix stored in the coupled bases (et_basis_change_matrix()) to change the basis of the operator to the new coupled basis.

In [10]:
d_uncoupled = d_evr.tensor(d_es)
d_baseB = d_uncoupled.change_basis(caseB_basis, caseB_basis.get_basis_change_matrix())

In [11]:
d_baseB.get_connected_states()

[(|elec=X, S=0.5, R=1, k=-1, J=0.5, mJ=-0.5>,
  |elec=A, S=0.5, R=1, k=0, J=1.5, mJ=-1.5>),
 (|elec=X, S=0.5, R=1, k=-1, J=0.5, mJ=0.5>,
  |elec=A, S=0.5, R=1, k=0, J=0.5, mJ=-0.5>),
 (|elec=X, S=0.5, R=1, k=-1, J=0.5, mJ=0.5>,
  |elec=A, S=0.5, R=1, k=0, J=1.5, mJ=-0.5>),
 (|elec=X, S=0.5, R=1, k=-1, J=1.5, mJ=-0.5>,
  |elec=A, S=0.5, R=1, k=0, J=1.5, mJ=-1.5>),
 (|elec=X, S=0.5, R=1, k=-1, J=1.5, mJ=0.5>,
  |elec=A, S=0.5, R=1, k=0, J=0.5, mJ=-0.5>),
 (|elec=X, S=0.5, R=1, k=-1, J=1.5, mJ=0.5>,
  |elec=A, S=0.5, R=1, k=0, J=1.5, mJ=-0.5>),
 (|elec=X, S=0.5, R=1, k=-1, J=1.5, mJ=1.5>,
  |elec=A, S=0.5, R=1, k=0, J=0.5, mJ=0.5>),
 (|elec=X, S=0.5, R=1, k=-1, J=1.5, mJ=1.5>,
  |elec=A, S=0.5, R=1, k=0, J=1.5, mJ=0.5>),
 (|elec=X, S=0.5, R=1, k=0, J=1.5, mJ=-1.5>,
  |elec=A, S=0.5, R=1, k=-1, J=0.5, mJ=-0.5>),
 (|elec=X, S=0.5, R=1, k=0, J=0.5, mJ=-0.5>,
  |elec=A, S=0.5, R=1, k=-1, J=0.5, mJ=0.5>),
 (|elec=X, S=0.5, R=1, k=0, J=1.5, mJ=-0.5>,
  |elec=A, S=0.5, R=1, k=-1, J=0.5, mJ=0.5>)

### Another example with ATM rotational basis

In [12]:
def wavenumber_to_Hz(k):
    return k * 299792458 * 100
def wavenumber_to_GHz(k):
    return k * 299792458 * 100 / 1e9

A = wavenumber_to_GHz(13.05744)
BC_avg2 = wavenumber_to_GHz(0.296652) # (B+C)/2
BC_diff4 = wavenumber_to_GHz(1.8894e-3) # (B-C)/4
rot_basis_ATM = ATM_RotationalBasis(A,BC_avg2,BC_diff4,R_range=(0,1),m_range=(-5,5))

evr_basis_ATM = vibronic_basis * rot_basis_ATM
d_evr_ATM = DipoleOperator_evr(evr_basis_ATM,vibronic_d, 1)

In [13]:
d_evr_ATM.get_connected_states()

[(|elec=X, R=0, k_a=0, k_c=0, mR=0>, |elec=A, R=1, k_a=1, k_c=1, mR=-1>),
 (|elec=X, R=1, k_a=1, k_c=1, mR=-1>, |elec=A, R=0, k_a=0, k_c=0, mR=0>),
 (|elec=X, R=1, k_a=0, k_c=1, mR=0>, |elec=A, R=1, k_a=1, k_c=0, mR=-1>),
 (|elec=X, R=1, k_a=1, k_c=0, mR=-1>, |elec=A, R=1, k_a=0, k_c=1, mR=0>),
 (|elec=X, R=1, k_a=0, k_c=1, mR=1>, |elec=A, R=1, k_a=1, k_c=0, mR=0>),
 (|elec=X, R=1, k_a=1, k_c=0, mR=0>, |elec=A, R=1, k_a=0, k_c=1, mR=1>),
 (|elec=X, R=1, k_a=1, k_c=1, mR=1>, |elec=A, R=0, k_a=0, k_c=0, mR=0>),
 (|elec=X, R=0, k_a=0, k_c=0, mR=0>, |elec=A, R=1, k_a=1, k_c=1, mR=1>),
 (|elec=X, R=1, k_a=1, k_c=0, mR=0>, |elec=A, R=1, k_a=0, k_c=1, mR=-1>),
 (|elec=X, R=1, k_a=0, k_c=1, mR=-1>, |elec=A, R=1, k_a=1, k_c=0, mR=0>),
 (|elec=X, R=1, k_a=1, k_c=0, mR=1>, |elec=A, R=1, k_a=0, k_c=1, mR=0>),
 (|elec=X, R=1, k_a=0, k_c=1, mR=0>, |elec=A, R=1, k_a=1, k_c=0, mR=1>)]

In [14]:
J_basis_ATM = evr_basis_ATM.couple(es_basis)
J_basis_ATM.rename_symbols("J","mJ")

d_uncoupled_ATM = d_evr_ATM.tensor(d_es)
d_caseB_ATM = d_uncoupled_ATM.change_basis(J_basis_ATM, J_basis_ATM.get_basis_change_matrix())

d_caseB_ATM.get_connected_states()

[(|elec=X, S=0.5, R=0, k_a=0, k_c=0, J=0.5, mJ=-0.5>,
  |elec=A, S=0.5, R=1, k_a=1, k_c=1, J=1.5, mJ=-1.5>),
 (|elec=X, S=0.5, R=0, k_a=0, k_c=0, J=0.5, mJ=0.5>,
  |elec=A, S=0.5, R=1, k_a=1, k_c=1, J=0.5, mJ=-0.5>),
 (|elec=X, S=0.5, R=0, k_a=0, k_c=0, J=0.5, mJ=0.5>,
  |elec=A, S=0.5, R=1, k_a=1, k_c=1, J=1.5, mJ=-0.5>),
 (|elec=X, S=0.5, R=1, k_a=1, k_c=1, J=1.5, mJ=-1.5>,
  |elec=A, S=0.5, R=0, k_a=0, k_c=0, J=0.5, mJ=-0.5>),
 (|elec=X, S=0.5, R=1, k_a=1, k_c=1, J=0.5, mJ=-0.5>,
  |elec=A, S=0.5, R=0, k_a=0, k_c=0, J=0.5, mJ=0.5>),
 (|elec=X, S=0.5, R=1, k_a=1, k_c=1, J=1.5, mJ=-0.5>,
  |elec=A, S=0.5, R=0, k_a=0, k_c=0, J=0.5, mJ=0.5>),
 (|elec=X, S=0.5, R=1, k_a=0, k_c=1, J=0.5, mJ=-0.5>,
  |elec=A, S=0.5, R=1, k_a=1, k_c=0, J=1.5, mJ=-1.5>),
 (|elec=X, S=0.5, R=1, k_a=0, k_c=1, J=0.5, mJ=0.5>,
  |elec=A, S=0.5, R=1, k_a=1, k_c=0, J=0.5, mJ=-0.5>),
 (|elec=X, S=0.5, R=1, k_a=0, k_c=1, J=0.5, mJ=0.5>,
  |elec=A, S=0.5, R=1, k_a=1, k_c=0, J=1.5, mJ=-0.5>),
 (|elec=X, S=0.5, R=1, k_

## Symmetry of the state

If state irreps and operator irreps are defined, you can check whether a transition is possible by checking if the product of the irreps are A1.

In [21]:
X.irrep * vibronic_d_b.irrep * A.irrep

C2v_A1 representation
e: [[1]]
C_2: [[1]]
σ_v(xz): [[1]]
σ_v(yz): [[1]]